# Calculating the skin friction coefficient, $C_f$

Skin friction coefficient is a non-dimensional quantity that characterises the friction drag that results from fluid viscosity and the presence of the wall. It is defined as the wall shear stress, $\tau_w$ divided by the dynamic pressure, $\rho U_b^2/2$:

$C_f=\frac{2\tau_w}{\rho U_b^2}$

with 

$\tau_w=\mu\frac{\partial \bar{\vec{u}}}{\partial \vec{n}}$

For many flows, $C_f$ exhibits a degree of similarity in particular with Reynolds number, $\rho U_b L/\mu$ and correlations exist for many flows, including turbulent pipe flows:

$C_f = \frac{1}{4}\left(-1.8\log_{10}\left(\frac{6.9}{Re_D}\right)\right)^{-2}$

## Turbulent pipe flow
In this tutorial, we use streamwise periodic pipe flow at $Re_D=19000$ ($Re_\tau=550$). This case is non-dimensionalised such that $\rho=1$, $U_b=1$, radius, $D=1$ and $\mu=1/Re_{D}$. Note that unlike in the channel case, $z$ is the streamwise direction

Using the correlation above $C_f=0.00652$. 

To decompress the data

```bash
gzip -d turbPipe/*.gz
```

## Calculating $\tau_w$
The key element of this is computing the wall shear stress
1. Calculate the derivatives
2. Extract the wall from the volume mesh
3. Compute the gradient normal to the wall
4. Average over the wall

In [3]:
import pyvista as pv
# Load the data
fluid_reader = pv.get_reader('turbPipe/turbPipe.nek5000')
fluid_reader.enable_merge_points()
fluid_reader.set_active_time_point(fluid_reader.number_time_points-1)

fluid = fluid_reader.read()
fluid.clip(normal='x').plot(window_size=(450,450),
                       scalars='Velocity',
                       cmap='bwr')

3D-Mesh found, spectral element of size = 8*8*8=512


Widget(value='<iframe src="http://localhost:38915/index.html?ui=P_0x7f9437443400_2&reconnect=auto" class="pyvi…

### compute derivatives and extract surface


In [4]:
fluid = fluid.compute_derivative(scalars='Velocity')

# extract all boundaries including inlet and outlet which we do not want
surface = fluid.extract_surface()
# get bounding box and shave off the inlet and outlet
xmin, xmax, ymin, ymax, zmin, zmax = surface.bounds
dz = 0.01*(zmax - zmin)
new_bounds = [xmin, xmax, ymin, ymax, zmin+dz, zmax-dz]

# clip the surface with the new bounding box with inlet and outlet shaved
wall = surface.clip_box(new_bounds, invert=False)

#plot
wall.plot(window_size=(450,450))


Widget(value='<iframe src="http://localhost:38915/index.html?ui=P_0x7f9437edb160_3&reconnect=auto" class="pyvi…

### Project gradient onto surface normals

In order the extract the surface normals we need to convert the wall from an `UnstructuredGrid` to a `PolyData`. Then we can use `compute_normals`. As only the point normals are required, `cell_normals=False` is passed. To calculate the magnitude of the velocity derivative normal to the wall we use

$\vert \frac{\partial u_z}{\partial \vec{n}}\vert \cos\theta =  \frac{\frac{\partial u_z}{\partial \vec{x}}\cdot \vec{n}}{\vert \vec{n}\vert},$

where $\vert n\vert = 1$.

In [5]:
import numpy as np

# extract cell normals
wall = wall.extract_geometry().compute_normals(cell_normals=False)
normals = wall.point_data['Normals']

# streamwise direction is z so we need ZX, ZY and ZZ from tensor
dudz = wall.point_data['gradient'][:,6:]

# compute dot product
du_n = np.linalg.vecdot(dudz, normals)

# Calculate average from all wall cells
dudn_mean = abs(du_n).mean()

### Compute $C_f$

In [6]:
ReD = 19000

mu = 1./ReD
Ub = 1.
rho = 1.

tau_w = mu*dudn_mean

Cf = 2*tau_w/(rho*Ub*Ub)
Cf_ref = 0.25*np.pow(-1.8*np.log10(6.9/ReD), -2)

print("Calculated: ", Cf)
print("Correlation: ", Cf_ref)


Calculated:  0.0068348483
Correlation:  0.006520820185665624
